# Preprocessing and storing DF in pickle
Perhaps we might Dwh Pat Dn Date later on. 

In [198]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
pd.set_option('display.max_columns', None)

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

In [199]:
bio = pd.read_excel('Dropbox/Healthcare work Patric/20170713_Marc Gliomes_Bio.xlsx', index_col = None)
clin = pd.read_excel('Dropbox/Healthcare work Patric/20170713_Marc Gliomes_Clin.xlsx', index_col = None)

In [200]:
b = bio.loc[bio['Pat Nip']==4406078178,'Dwh Prv Date Chir']
c = clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']

#c.iloc[:len(c)]=1
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] = c
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] 
#print(clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'].iloc[i])
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']    
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())
#bio.loc[2,'Dwh Prv Date Chir'].to_pydatetime()-timedelta(days=1)
#clin['Clin Chir Date'] = clin['Clin Chir Date'].apply(lambda x: x.to_pydatetime() + timedelta(days=1))
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())

Fix the surgery and clinical date so that they match

In [201]:
# Make the two surgery dates match 
# You are taking copies of the dataframe with loc and iloc, so you need to pay attention when you do assignment
# Can definitely remove one or both 'for' loops
for val in np.unique(clin['Pat Nip']): 
    bio_data = bio.loc[bio['Pat Nip']==val,'Dwh Prv Date Chir']
    clin_data = clin.loc[clin['Pat Nip']==val,'Clin Chir Date']
    lc = len(clin_data)
    lb = len(bio_data)
    if lc <=lb:
        for i in range(0,lc):
            clin_data.iloc[i] = bio_data.iloc[i]        
    else: 
        for i in range(0,lb):
            clin_data.iloc[i] = bio_data.iloc[i]
    clin.loc[clin['Pat Nip']==val,'Clin Chir Date'] = clin_data        

In [202]:
df = pd.merge(bio, clin, left_on = ['Pat Nip','Dwh Prv Date Chir'], right_on = ['Pat Nip','Clin Chir Date'],how='outer')
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

Before merging sexes, birthdays and deaths

In [203]:
# Can check before merging the whole thing as well. 
#bio['Pat Sexe'].isnull().value_counts()
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())

False    7456
True      369
Name: Pat Sexe_x, dtype: int64
False    7725
True      100
Name: Pat Date Naissance_x, dtype: int64
True     4921
False    2904
Name: Dwh Pat Date Deces_x, dtype: int64


# Copy all _y columns into _x: sex, birthday and death, clinical histology, grade histology

In [204]:
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Sexe_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Sexe_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Sexe_x'] = x_data

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Date Naissance_x'] = x_data    

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x'] = x_data   
    
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie']
    y_data = df.loc[df['Pat Nip']==val,'Clin Diag Histologie']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Grade']
    y_data = df.loc[df['Pat Nip']==val,'Grade histo']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Grade'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x'] = x_data   


In [205]:
# A small improvement
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())
print(df['Dwh Prv Diag Histologie'].isnull().value_counts())

False    7493
True      332
Name: Pat Sexe_x, dtype: int64
False    7762
True       63
Name: Pat Date Naissance_x, dtype: int64
True     4908
False    2917
Name: Dwh Pat Date Deces_x, dtype: int64
False    7825
Name: Dwh Prv Diag Histologie, dtype: int64


In [206]:
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

In [207]:
df.rename(columns = {'Pat Nip':'ID','Pat Sexe_x':'Sex','Dwh Pat Dn Date_x':'Dernieres_nouvelles','Pat Date Naissance_x':'DoB','Dwh Pat Date Deces_x':'Death_date',
                      'Dwh Prv Date Chir':'Surgery_date', 'Dwh Prv Diag Histologie':'Tumor_type','Grade':'Tumor_grade',
                      'Dwh Gen Idh1':'Gene_Idh1', 'Dwh Gen Idh2':'Gene_Idh2','Dwh Gen Htert C228T':'Gene_Htert_C228T', 
                      'Dwh Gen Htert C250T':'Gene_Htert_C250T', "Dwh Gen P53":'Gene_P53', 'Dwh Gen Mgmt':'Gene_Mgmt', 
                      'Dwh Gen Egfr':'Gene_Egfr', 'Dwh Gen Mdm2':'Gene_Mdm2','Dwh Gen Cdk4':'Gene_Cdk4',
                      'Dwh Gen P16':'Gene_P16', 'Dwh Ihc Atrx':'Gene_Ihc_Atrx', 'Dwh Chr Ch1P':'Gene_Ch1P',
                      'Dwh Chr Ch19Q':'Gene_Ch19Q', 'Dwh Chr Ch10P':'Gene_Ch10P', 'Dwh Chr Ch10Q':'Gene_Ch10Q', 'Dwh Chr Ch9P':'Gene_Ch9P',
                      'Dwh Chr Ch9Q':'Gene_Ch9Q', 'Dwh Pat Tumeur Siege':'Tumor_Location', 'Dwh Pat Tumeur Position':'Tumor_Position',
                      'Clin Evolution':'Clinical_evolution', 'Clin Type Chirurgie':'Surgery_type',
                      'Clin Rx Date':'Rx_date', 'Dwh Chm Date Debut':'Chemo_start_date'}, inplace = True)
df.drop(['Dwh Pat Dn Date_y','Pat Sexe_y','Dwh Pat Date Deces_y','Pat Date Naissance_y'], axis=1,inplace=True)
df.drop('Dwh Pat Consentement', axis=1, inplace=True)
df.drop('Dwh Prv Nip', axis=1, inplace=True)
df.drop(['Clin Diag Histologie', 'Grade histo','Clin Chir Date',],axis=1,inplace=True)

In [208]:
df.rename(columns = {'Sex':'Gender'}, inplace = True) 
#write_to_pickle(df,'post_merge_data')
df.columns

Index(['ID', 'Gender', 'DoB', 'Dernieres_nouvelles', 'Death_date',
       'Surgery_date', 'Tumor_type', 'Tumor_grade', 'Gene_Idh1', 'Gene_Idh2',
       'Gene_Htert_C228T', 'Gene_Htert_C250T', 'Gene_P53', 'Gene_Mgmt',
       'Gene_Egfr', 'Gene_Mdm2', 'Gene_Cdk4', 'Gene_P16', 'Gene_Ihc_Atrx',
       'Gene_Ch1P', 'Gene_Ch19Q', 'Gene_Ch10P', 'Gene_Ch10Q', 'Gene_Ch9P',
       'Gene_Ch9Q', 'Tumor_Location', 'Tumor_Position', 'Clinical_evolution',
       'Surgery_type', 'Rx_date', 'Chemo_start_date'],
      dtype='object')

In [17]:
#df = read_from_pickle('post_merge_data')

In [209]:
df.head(2)

,ID,Gender,DoB,Dernieres_nouvelles,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date
0,4,M,1940-05-22,NaN,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT
1,5,M,1947-10-15,NaN,1999-12-15,1999-05-29,glioblastome IV,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sus-T hémisp,G,TT ini,biopsie,1999-07-12,1999-06-29


# End of merging. Next we deal with nan/empty values

In [210]:
df_clean = df.copy(deep=True)

In [211]:
df_clean = df.copy(deep=True)

#Marc comment: if a patient misses data for both IDH1 and IDH2, poor quality record = delete 
# Retain observations for which either Idh1 or Idh2 are null but not both
df_clean = df_clean.loc[(df_clean.Gene_Idh2.notnull()) | (df_clean.Gene_Idh1.notnull())]

df_clean= df_clean[df_clean.DoB.notnull()]
df_clean = df_clean[df_clean.Gender.notnull()]

# Remove dernieres nouvelles as in this notebook we focus only on patients with a confirmed survival time since surgery
df_clean.drop('Dernieres_nouvelles', axis=1, inplace=True)

In [212]:
# Sanity check
(df_clean.Gene_Idh1.isnull() & df_clean.Gene_Idh2.isnull()).value_counts()

False    7057
dtype: int64

In [213]:
# Surgery date is the beginning of the survival period in our study
# We therefore cannot work with missing values here (beyond specific left censoring methods of analysis)
# Remove NaN for surgery dates before creating new variable
df_clean = df_clean[df_clean.Surgery_date.notnull()]
df_clean.Surgery_date.isnull().value_counts()

False    6794
Name: Surgery_date, dtype: int64

In [214]:
# Add age at surgery variable
df_clean['Age_surgery'] = df_clean['Surgery_date'] - df_clean['DoB']

In [215]:
len(df_clean)

6794

In [216]:
# Marc comment: keep only observations where Clinical evolution was "TT ini" (initial evaluation)
df_clean = df_clean.loc[df_clean['Clinical_evolution'] == "TT ini"]

In [217]:
len(df_clean)

3758

# Time manipulations

In [218]:
df_clean.Death_date.isnull().value_counts()

False    2152
True     1606
Name: Death_date, dtype: int64

In [219]:
# Convert Age at surgery to years
df_clean['Age_surgery'] = df_clean['Age_surgery'] / np.timedelta64(1, 'Y')
df_clean['Age_surgery'] = df_clean['Age_surgery'].round()

In [220]:
# Keep only patients that have a confirmed death date
# Comment the first line to have the second approach
df_clean = df_clean[df_clean.Death_date.notnull()]
df_clean['life_expectancy'] = df_clean.Death_date - df_clean.Surgery_date
df_clean['life_expectancy'] = df_clean['life_expectancy'] / np.timedelta64(1, 'D')

In [221]:
df_clean_test = df_clean.copy(deep=True)

In [222]:
#Marc rule: aggregate "glioblastoma avec oligo IV" into "GBM"; "astrocytome gémistocytique (=A2)" into "astrocytoma grade II"
#we also remove grades from tumor types, since they are contained in "Tumor_grade" variable
df_clean_test['Tumor_type'].replace({"glioblastome IV": "GBM", "oligo.haut grade III":"oligo",
                                "gliome mixte ana III":"gliome", "oligo. bas grade II": "oligo",
                                "glioblastome avec oligo. IV": "GBM", "gliome mixte II": "gliome",
                                "astrocytome anaplasique III": "astrocytome", "astrocytome bas grade II":"astrocytome",
                                "astrocytome gémistocytique (=A2":"astrocytome", "gangliogliome anaplasique III":"gangliogliome anaplasique"}, inplace=True)
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non précisé,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,sus-T hémisp,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,sus-T hémisp,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,D,TT ini,exérèse,NaT,NaT,58.0,433.0
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,non précisé,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,non précisé,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0
12,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992-03-01,NaT,61.0,316.0
14,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,PERDU,PERDU,PARTIEL,NON PERTE,sus-T hémisp,G,TT ini,partielle,1993-08-11,1993-08-05,43.0,571.0


In [223]:
df_clean_test['Tumor_type'].value_counts()

GBM                                 1229
oligo                                431
gliome                               262
astrocytome                          192
gangliogliome                         14
astrocytome gémistocytique (=A2)      14
astrocytome pilocytique               10
Name: Tumor_type, dtype: int64

In [224]:
#Marc comment recode Tumor_Location
# A-sus-T hémisp            3550
# A-sus-T hémisp+profond     524
# N-non précisé              257
# A-sus-T profond            209
# A-Frontal                  134
# B-tronc cérébral            32
# B-cervelet                  31
# C-ventriculaire             20
# autre                     18
# B-moelle/rachis             10
# A-Rolandique                 3

df_clean_test['Tumor_Location'].replace({"sus-T hémisp": "A", "sus-T hémisp+profond":"A",
                                "non précisé":"N", "sus-T profond": "A",
                                "Frontal": "A", "tronc cérébral": "B",
                                "cervelet": "B", "ventriculaire":"C",
                                "moelle/rachis":"B", "Rolandique":"A"}, inplace=True)
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0
12,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,A,G,TT ini,partielle,1992-03-01,NaT,61.0,316.0
14,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,PERDU,PERDU,PARTIEL,NON PERTE,A,G,TT ini,partielle,1993-08-11,1993-08-05,43.0,571.0


In [225]:
# implementing rule per Marc comments
# collapse'Gene_Idh1', 'Gene_Idh2' into "IDH"
# IDH mutant = 'Gene_Idh1' OR 'Gene_Idh2' mutant
# IDHwt = 'Gene_Idh1'wt AND  'Gene_Idh2'wt; OR 'Gene_Idh1'wt AND IDH2 unknown
# Expl : IDH2 mutation are much less frequent than IDH1 mutations (5% of all the mutations)

df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE'), "IDH"] = "mutant"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'] == 'NORMAL'), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'].isnull()), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'] == "NC"), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NC') & (df_clean_test['Gene_Idh2'] == 'NC'), "IDH"] = "NC"
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0,wt
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0,wt
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0,wt
12,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,A,G,TT ini,partielle,1992-03-01,NaT,61.0,316.0,wt
14,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,PERDU,PERDU,PARTIEL,NON PERTE,A,G,TT ini,partielle,1993-08-11,1993-08-05,43.0,571.0,wt


In [226]:
# implementing rule per Marc comments
# collapse 'Gene_Htert_C228T', 'Gene_Htert_C250T' into TERT
# Tert mutant= 'Gene_Htert_C228T'mut OR 'Gene_Htert_C250T'mut
# Tert wt= 'Gene_Htert_C228T'wt  AND 'Gene_Htert_C250T' wt
df_clean_test.loc[(df_clean_test['Gene_Htert_C228T'] == 'ALTERE') | (df_clean_test['Gene_Htert_C250T'] == 'ALTERE'), "TERT"] = "mutant"
df_clean_test.loc[(df_clean_test['Gene_Htert_C228T'] == 'NORMAL') & (df_clean_test['Gene_Htert_C250T'] == 'NORMAL'), "TERT"] = "wt"
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH,TERT
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant,wt
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC,NaN
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt,mutant
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt,wt
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt,mutant
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0,wt,mutant
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0,wt,wt
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0,wt,mutant
12,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,A,G,TT ini,partielle,1992-03-01,NaT,61.0,316.0,wt,mutant
14,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,PERDU,PERDU,PARTIEL,NON PERTE,A,G,TT ini,partielle,1993-08-11,1993-08-05,43.0,571.0,wt,mutant


In [227]:
# Marc comment: implementing clinical interaction variable between IDH and TERT
# we will implement this rule based on the underlying IDH1, IDH2, TERT C250T and TERT C228T because they contain more information
# for example if IDH1 is missing, and IDH 2 is NORMAL, IDH will also be missing even though IDH2 is present 
# IDH+ TERT+ == 1 (very good prognosis)
# IDH+ TERT- == 2
# IDH- TERT- == 3
# IDH- TERT+ == 4

df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE')) & ((df_clean_test['Gene_Htert_C228T'] == "ALTERE") | (df_clean_test['Gene_Htert_C250T'] == "ALTERE")), "IDH_TERT"] = 1; df_clean_test
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE')) & ((df_clean_test['Gene_Htert_C228T'] == "NORMAL") | (df_clean_test['Gene_Htert_C250T'] == "NORMAL")), "IDH_TERT"] = 2; df_clean_test
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'NORMAL') | (df_clean_test['Gene_Idh2'] == 'NORMAL')) & ((df_clean_test['Gene_Htert_C228T'] == "NORMAL") | (df_clean_test['Gene_Htert_C250T'] == "NORMAL")), "IDH_TERT"] = 3; df_clean_test
df_clean_test.loc[((df_clean_test['Gene_Idh1'] == 'NORMAL') | (df_clean_test['Gene_Idh2'] == 'NORMAL')) & ((df_clean_test['Gene_Htert_C228T'] == "ALTERE") | (df_clean_test['Gene_Htert_C250T'] == "ALTERE")), "IDH_TERT"] = 4; df_clean_test

# ignore below implementation based on IDH and TERT directly
# df_clean_test.loc[(df_clean_test['IDH'] == 'mutant') & (df_clean_test['TERT'] == 'mutant'), "IDH_TERT"] = 1
# df_clean_test.loc[(df_clean_test['IDH'] == 'mutant') & (df_clean_test['TERT'] == 'wt'), "IDH_TERT"] = 2
# df_clean_test.loc[(df_clean_test['IDH'] == 'wt') & (df_clean_test['TERT'] == 'wt'), "IDH_TERT"] = 3
# df_clean_test.loc[(df_clean_test['IDH'] == 'wt') & (df_clean_test['TERT'] == 'mutant'), "IDH_TERT"] = 4

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant,wt,3.0
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC,NaN,NaN
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt,mutant,4.0
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt,wt,3.0
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt,mutant,4.0
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0,wt,mutant,4.0
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0,wt,wt,3.0
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0,wt,mutant,4.0
12,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,A,G,TT ini,partielle,1992-03-01,NaT,61.0,316.0,wt,mutant,4.0
14,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,PERDU,PERDU,PARTIEL,NON PERTE,A,G,TT ini,partielle,1993-08-11,1993-08-05,43.0,571.0,wt,mutant,4.0


In [228]:
#now delete IDH1 and IDH2 variables
df_clean_test.drop("Gene_Idh1", axis = 1, inplace=True)
df_clean_test.drop("Gene_Idh2", axis = 1, inplace=True)

In [229]:
#now delete TERT variables C228T and C250T
df_clean_test.drop("Gene_Htert_C228T", axis = 1, inplace=True)
df_clean_test.drop("Gene_Htert_C250T", axis = 1, inplace=True)

In [230]:
# Marc comment: implementing clinical 1p19q codeletion variable
# 1p19q codel instead of 'Gene_Ch1P', 'Gene_Ch19Q'= in fact not relevant when separated (consider complete loss, 
# no partial loss) = What is important is : 1p19q codélétion and this is always associated with IDH
# 1p19q codélétion = 'Gene_Ch1P loss AND  'Gene_Ch19Q loss AND IDH mutant (see above)

# if Ch1P is "PERDU" AND CH19Q is PERDU, then 1p19q_codel = 1
df_clean_test.loc[(df_clean_test['Gene_Ch1P'] == 'PERDU') & (df_clean_test['Gene_Ch19Q'] == 'PERDU'), "1p19q_codel"] = 1

#problem with following command is that if one value is missing, it will also impute 0 instead of keeping it as missing
#df_clean_test.loc[((df_clean_test['Gene_Ch1P'] != 'PERDU') | (df_clean_test['Gene_Ch19Q'] != 'PERDU'), "1p19q_codel"] = 0

# if ((Ch1P is not null) AND (CH19Q is not null)) AND ((Ch1Q != PERDU) OR (Ch19Q != PERDU)) then 1p19q_codel is 0

df_clean_test.loc[((df_clean_test['Gene_Ch1P'].notnull()) & (df_clean_test['Gene_Ch19Q'].notnull()) & ((df_clean_test['Gene_Ch1P'] != 'PERDU') | (df_clean_test['Gene_Ch19Q'] != 'PERDU'))), "1p19q_codel"] = 0

In [231]:
df_clean_test['1p19q_codel'].isnull().value_counts()

True     1159
False     993
Name: 1p19q_codel, dtype: int64

In [232]:
# Marc comment: now delete 1P and 19Q since alone they are each insignificant
df_clean_test.drop("Gene_Ch1P", axis = 1, inplace=True)
df_clean_test.drop("Gene_Ch19Q", axis = 1, inplace=True)

In [233]:
# Marc comment: If IDH is missing, the tumor type is GBM, and the patient is >55 y, the probability of 
# IDHmutation is around 5%, so you may also consider to fill this gap
# we will use age at surgery here for this purpose
# we will impute both IDH missing and IDH NC (non conclusive) based on this rule

df_clean_test.loc[((df_clean_test['IDH'].isnull()) & (df_clean_test['Tumor_type'] == "GBM") & (df_clean_test['Age_surgery'] > 55.0)), "IDH"] = "wt"
df_clean_test.loc[((df_clean_test['IDH'] == "NC") & (df_clean_test['Tumor_type'] == "GBM") & (df_clean_test['Age_surgery'] > 55.0)), "IDH"] = "wt"

In [234]:
# Marc comment: delete Dwh Chr CH9Q
df_clean_test.drop("Gene_Ch9Q", axis = 1, inplace=True)

In [235]:
# Marc comment: recode CH9P
df_clean_test['Gene_Ch9P'].replace({"PARTIEL":"PERTE", "PERDU": "PERTE"}, inplace=True)

In [236]:
# sanity check of above recoding
df_clean_test['Gene_Ch9P'].value_counts()

NON PERTE    503
PERTE        460
GAIN          30
Name: Gene_Ch9P, dtype: int64

In [237]:
# Marc comment: for Ch10Q, consider just instance where = PERDU
# create binary variable PERTE and NON PERTE therefore
df_clean_test['Gene_Ch10Q'].replace({"PERDU":"PERTE", "PARTIEL": "NON PERTE", "GAIN": "NON PERTE"}, inplace=True)

In [238]:
# sanity check of above
df_clean_test['Gene_Ch10Q'].value_counts()

PERTE        544
NON PERTE    466
Name: Gene_Ch10Q, dtype: int64

In [239]:
# Marc comment: delete Ch10P
df_clean_test.drop("Gene_Ch10P", axis = 1, inplace=True)

In [240]:
# Marc comment: IDHmut-Tertmut are 1p19codel in 90% of the cases
# impute missing 1p19q codel values per above
df_clean_test.loc[(df_clean_test['IDH_TERT'] == 1) & (df_clean_test['1p19q_codel'].isnull()), "1p19q_codel"] = 1

In [241]:
# Marc comment: IDHmut-Tertwt are ATRX mut in 90% of the cases
# impute missing ATRX values per above
df_clean_test.loc[(df_clean_test['IDH_TERT'] == 2) & (df_clean_test['Gene_Ihc_Atrx'].isnull()), "Gene_Ihc_Atrx"] = "extinction"

In [242]:
# Marc comment: EGFR amplification is extremely rare in IDH mutant
# impute missing EGFR values per above
df_clean_test.loc[(df_clean_test['IDH'] == "mutant") & (df_clean_test['Gene_Egfr'].isnull()), "Gene_Egfr"] = "NORMAL"

In [243]:
# Marc comment: EGFR amplification never occurs in 1p19q codel
#impute missing EGFR values per above
df_clean_test.loc[(df_clean_test['1p19q_codel'] == 1) & (df_clean_test['Gene_Egfr'].isnull()), "Gene_Egfr"] = "NORMAL"

In [244]:
# Marc comment: Loss of chr 10q is rarely associated with IDH mutant (<5%) and almost never (1%) with 1p19q codel
df_clean_test.loc[(df_clean_test['IDH'] == "mutant") & (df_clean_test['Gene_Ch10Q'].isnull()), "Gene_Ch10Q"] = "NON PERTE"
df_clean_test.loc[(df_clean_test['1p19q_codel'] == 1) & (df_clean_test['Gene_Ch10Q'].isnull()), "Gene_Ch10Q"] = "NON PERTE"

In [245]:
# We keep only the first record of patients with multiple surgeries
df_clean_test['ID'].duplicated().value_counts()

False    2086
True       66
Name: ID, dtype: int64

In [246]:
# Sort by ID and Surgery date, in order to keep only earliest surgery date
df_clean_test.sort_values(['ID', 'Surgery_date'], inplace=True)

In [247]:
# Indicate whether duplicate and check sum of them
df_clean_test['multiple_surgeries'] = df_clean_test.duplicated(['ID'])
df_clean_test['multiple_surgeries'].sum()

66

In [248]:
# Reduce to observations that don't have multiple surgeries

df_clean_test.drop_duplicates('ID',keep='first',inplace=True)
df_clean_test.head(5)

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT,1p19q_codel,multiple_surgeries
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,PERTE,NON PERTE,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant,wt,3.0,0.0,False
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC,NaN,NaN,NaN,False
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PERTE,PERTE,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt,mutant,4.0,0.0,False
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,PERTE,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt,wt,3.0,0.0,False
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt,mutant,4.0,0.0,False


In [249]:
df_final = df_clean_test.copy(deep=True)
df_final.multiple_surgeries.isnull().value_counts()

False    2086
Name: multiple_surgeries, dtype: int64

In [250]:
# They seem to have been datetime objects without the below conversions?
# See here why we need to use loc: 
# https://stackoverflow.com/questions/26724378/pandas-settingwithcopywarning?noredirect=1&lq=1

df_final.loc[:,'Rx_date'] = pd.to_datetime(df_final.Rx_date,errors = 'coerce')
df_final.loc[:,'Chemo_start_date'] = pd.to_datetime(df_final.Chemo_start_date,errors = 'coerce')

df_final.loc[:,'Rx_date'] = df_final['Rx_date'].dt.year
df_final.loc[:,'Chemo_start_date']=df_final.Chemo_start_date.dt.year

In [37]:
# Useful
#for col in df_clean.columns:
 #    print('{0}\n  {1}\n'.format(col,df_clean[col].isnull().value_counts()))

# Merge KPS dataset

In [251]:
# Import KPS dataset to merge it
kps = pd.read_excel('Dropbox/Healthcare work Patric/KPS.xlsx')

In [252]:
# Sort by ID and surgery date, as above
kps.sort_values(['NIP', 'CHIR_DATE'],inplace=True)

In [253]:
# Reduce to single observations as with multiple surgeries from previous dataset
kps.drop_duplicates('NIP',keep='first',inplace=True)

In [254]:
kps.shape

(6772, 6)

In [255]:
# Keep only columns that are of interest or needed for the merge
kps_sorted_final = kps[['NIP', 'ID_EC', 'IK_CLINIQUE', 'CHIR_DATE', 'CHIR_IK_PRE', 'RX_IK_PRE']]

In [256]:
# Merge the two datasets using surgery date and chirurgie date, only the later is not a datetime.
# Use ID for merge as well 
kps.loc[:,'CHIR_DATE'] = pd.to_datetime(kps.CHIR_DATE, errors = 'coerce')

In [295]:
df_final_kps = pd.merge(df_final, kps, how = 'left', left_on = ['ID', 'Surgery_date'], right_on = ['NIP', 'CHIR_DATE'])

In [296]:
len(df_final)

2086

In [297]:
len(df_final_kps)

2086

In [260]:
df_final_kps

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT,1p19q_codel,multiple_surgeries,NIP,ID_EC,CHIR_DATE,IK_CLINIQUE,CHIR_IK_PRE,RX_IK_PRE
0,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,PERTE,NON PERTE,A,G,TT ini,partielle,1992.0,1992.0,42.0,30.0,mutant,wt,3.0,0.0,False,1.000000e+01,TTini,1992-04-15,NaN,NaN,NaN
1,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986.0,NaN,28.0,2069.0,NC,NaN,NaN,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN
2,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PERTE,PERTE,A,G,TT ini,aucune,1990.0,NaN,59.0,393.0,wt,mutant,4.0,0.0,False,1.200000e+01,TTini,1990-02-13,NaN,NaN,70.0
3,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,PERTE,NaN,A,D,TT ini,partielle,1989.0,NaN,61.0,398.0,wt,wt,3.0,0.0,False,1.300000e+01,TTini,1989-11-17,NaN,60.0,NaN
4,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,PERTE,NON PERTE,A,D,TT ini,exérèse,NaN,NaN,58.0,433.0,wt,mutant,4.0,0.0,False,1.400000e+01,TTini,1992-01-22,NaN,70.0,NaN
5,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PERTE,NON PERTE,A,G,TT ini,exérèse,1991.0,NaN,56.0,143.0,wt,mutant,4.0,0.0,False,1.500000e+01,TTini,1991-10-23,NaN,80.0,80.0
6,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PERTE,NON PERTE,N,G,TT ini,exérèse,1990.0,NaN,69.0,408.0,wt,wt,3.0,0.0,False,NaN,NaN,NaT,NaN,NaN,NaN
7,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,PERTE,NON PERTE,N,non précisé,TT ini,exérèse,NaN,NaN,66.0,7.0,wt,mutant,4.0,0.0,False,1.700000e+01,TTini,1990-03-28,NaN,NaN,NaN
8,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome,3.0,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,A,G,TT ini,partielle,1992.0,NaN,61.0,316.0,wt,mutant,4.0,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN
9,22,M,1950-05-10,1995-01-29,1993-07-07,GBM,4.0,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PERTE,PERTE,A,G,TT ini,partielle,1993.0,1993.0,43.0,571.0,wt,mutant,4.0,0.0,False,2.200000e+01,TTini,1993-07-07,NaN,80.0,90.0


In [298]:
# Clinical_evolution has no missing values. ID_EC is a duplicate of Clinical_evolution, from KPS dataset - let's delete it
df_final_kps.drop("ID_EC", axis = 1, inplace=True)

In [299]:
# Marc comment: IK = take just one value : 'CHIR_IK_PRE', and if CHIR_IK_PRE is missing, take the average of the two others
df_final_kps['IK'] = df_final_kps['CHIR_IK_PRE']
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].notnull()) & (df_final_kps['RX_IK_PRE'].notnull()), 'IK'] = (df_final_kps['IK_CLINIQUE'] + df_final_kps['RX_IK_PRE'])/2
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].isnull()) & (df_final_kps['RX_IK_PRE'].notnull()), 'IK'] = df_final_kps['RX_IK_PRE'] 
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].notnull()) & (df_final_kps['RX_IK_PRE'].isnull()), 'IK'] = df_final_kps['IK_CLINIQUE'] 

In [300]:
# now drop all IK variables except for "IK"
df_final_kps.drop('CHIR_IK_PRE', axis =1, inplace = True)
df_final_kps.drop('IK_CLINIQUE', axis = 1, inplace = True)
df_final_kps.drop('RX_IK_PRE', axis = 1, inplace = True)

# Looks good so far 

In [301]:
# Code Rx date into either 1 if date is present, or leave it as missing otherwise
df_final_kps['RX'] = np.where(df_final_kps['Rx_date'].notnull(), 1, np.NaN)

In [302]:
df_final_kps['CHEM'] = np.where(df_final_kps['Chemo_start_date'].notnull(), 1, np.NaN)

In [303]:
#drop NIP as it's duplicate of ID after the merge
#drop multiple surgeries as it was only needed to remove duplicates
#drop CHIR Date as it comes from KPS dataset after merge and is duplicate of Surgery_date
#drop Rx_date and Chemo_start_date
#Final dataset before imputation
df_final_kps.drop('NIP', axis = 1, inplace=True)
df_final_kps.drop('multiple_surgeries', axis = 1, inplace=True)
df_final_kps.drop('CHIR_DATE', axis = 1, inplace=True)
df_final_kps.drop('Chemo_start_date', axis = 1, inplace=True)
df_final_kps.drop('Rx_date', axis = 1, inplace=True)

In [304]:
len(df_final_kps.columns)

29

In [307]:
#Drop death date
#Drop Dernieres nouvelles
#Drop surgery date
#drop clinical evolution
#drop DoB
df_final_kps.drop('Death_date', axis = 1, inplace=True)
df_final_kps.drop('Surgery_date', axis = 1, inplace=True)
df_final_kps.drop('Dernieres_nouvelles', axis = 1, inplace=True)
df_final_kps.drop('Clinical_evolution', axis = 1, inplace=True)
df_final_kps.drop('DoB', axis = 1, inplace=True)

In [308]:
len(df_final_kps.columns)

25

In [309]:
df_final_kps.head(2)

,ID,Gender,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Surgery_type,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT,1p19q_codel,IK,RX,CHEM
0,10,M,gliome,3.0,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,PERTE,NON PERTE,A,G,partielle,42.0,30.0,mutant,wt,3.0,0.0,NaN,1.0,1.0
1,11,M,astrocytome,2.0,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,exérèse,28.0,2069.0,NC,NaN,NaN,NaN,NaN,1.0,NaN


In [310]:
#df_final_kps['life_expectancy'] = pd.to_datetime(df_final_kps['life_expectancy'])
df_final_kps['life_expectancy'] = df_final_kps['life_expectancy'].round()

# Create Imputation dataset

In [55]:
#df_final_kps.drop(['DoB','Death_date','Surgery_date','Chemo_start_date','Rx_date'],axis=1,inplace=True)
#write_to_pickle(df_final_kps,"preimputation")

In [311]:
# Drop ID so that MICE imputation works
df_final_kps.drop(['ID'], axis=1,inplace=True)

In [ ]:
#checking how many observations we have with different numbers of NA data across variables
no_NAN = df_final_kps.dropna(thresh = 35)
len(no_NAN)

In [269]:
len(df_final_kps.columns)

33

In [285]:
df_final_kps

,Gender,Tumor_type,Tumor_grade,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch10Q,Gene_Ch9P,Tumor_Location,Tumor_Position,Surgery_type,Age_surgery,life_expectancy,IDH,TERT,IDH_TERT,1p19q_codel,IK,RX,CHEM
0,M,gliome,3.0,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,PERTE,NON PERTE,A,G,partielle,42.0,30.0,mutant,wt,3.0,0.0,NaN,1.0,1.0
1,M,astrocytome,2.0,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,exérèse,28.0,2069.0,NC,NaN,NaN,NaN,NaN,1.0,NaN
2,F,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PERTE,PERTE,A,G,aucune,59.0,393.0,wt,mutant,4.0,0.0,70.0,1.0,NaN
3,M,GBM,4.0,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,PERTE,NaN,A,D,partielle,61.0,398.0,wt,wt,3.0,0.0,60.0,1.0,NaN
4,F,GBM,4.0,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,PERTE,NON PERTE,A,D,exérèse,58.0,433.0,wt,mutant,4.0,0.0,70.0,NaN,NaN
5,M,GBM,4.0,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PERTE,NON PERTE,A,G,exérèse,56.0,143.0,wt,mutant,4.0,0.0,80.0,1.0,NaN
6,M,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PERTE,NON PERTE,N,G,exérèse,69.0,408.0,wt,wt,3.0,0.0,NaN,1.0,NaN
7,M,GBM,4.0,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,PERTE,NON PERTE,N,non précisé,exérèse,66.0,7.0,wt,mutant,4.0,0.0,NaN,NaN,NaN
8,F,astrocytome,3.0,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,A,G,partielle,61.0,316.0,wt,mutant,4.0,NaN,NaN,1.0,NaN
9,M,GBM,4.0,NORMAL,METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PERTE,PERTE,A,G,partielle,43.0,571.0,wt,mutant,4.0,0.0,80.0,1.0,1.0


In [315]:
df_final_kps.corr()

,Tumor_grade,Age_surgery,life_expectancy,IDH_TERT,1p19q_codel,IK,RX,CHEM
Tumor_grade,1.000000,0.483171,-0.524259,0.316438,-0.270721,-0.252534,NaN,NaN
Age_surgery,0.483171,1.000000,-0.435789,0.367233,-0.092515,-0.293575,NaN,NaN
life_expectancy,-0.524259,-0.435789,1.000000,-0.222966,0.401084,0.307808,NaN,NaN
IDH_TERT,0.316438,0.367233,-0.222966,1.000000,0.088009,-0.082236,NaN,NaN
1p19q_codel,-0.270721,-0.092515,0.401084,0.088009,1.000000,0.096139,NaN,NaN
IK,-0.252534,-0.293575,0.307808,-0.082236,0.096139,1.000000,NaN,NaN
RX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
df_final_kps['RX'].describe()

count    1473.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: RX, dtype: float64

In [320]:
len(df_final_kps.columns)

24

In [322]:
df_final_kps.columns

Index(['Gender', 'Tumor_type', 'Tumor_grade', 'Gene_P53', 'Gene_Mgmt',
       'Gene_Egfr', 'Gene_Mdm2', 'Gene_Cdk4', 'Gene_P16', 'Gene_Ihc_Atrx',
       'Gene_Ch10Q', 'Gene_Ch9P', 'Tumor_Location', 'Tumor_Position',
       'Surgery_type', 'Age_surgery', 'life_expectancy', 'IDH', 'TERT',
       'IDH_TERT', '1p19q_codel', 'IK', 'RX', 'CHEM'],
      dtype='object')

In [323]:
df_final_kps.drop("RX", axis = 1, inplace=True)
df_final_kps.drop("CHEM", axis = 1, inplace=True)

# Add it to a csv ready for imputation. Then read the imputed one for one last manipulation

In [324]:
#export dataset for imputation
df_final_kps.to_csv('Dropbox/Healthcare work Patric/imputation_dataset_no_censoring_16022018.csv', index=False)